<a href="https://colab.research.google.com/github/ykalathiya-2/AutoGluon/blob/main/knot_theory/knot_theory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoGluon Tabular - Quick Start

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/autogluon/autogluon/blob/stable/docs/tutorials/tabular/tabular-quick-start.ipynb)
[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/autogluon/autogluon/blob/stable/docs/tutorials/tabular/tabular-quick-start.ipynb)

In this tutorial, we will see how to use AutoGluon's `TabularPredictor` to predict the values of a target column based on the other columns in a tabular dataset.

Begin by making sure AutoGluon is installed, and then import AutoGluon's `TabularDataset` and `TabularPredictor`. We will use the former to load data and the latter to train models and make predictions.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!python -m pip install --upgrade pip
!python -m pip install --quiet autogluon

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'nvidia-ml-py3' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'nvidia-ml-py3'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'seqeval' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible repla

In [3]:
from autogluon.tabular import TabularDataset, TabularPredictor

## Example Data

For this tutorial we will use a dataset from the cover story of [Nature issue 7887](https://www.nature.com/nature/volumes/600/issues/7887): [AI-guided intuition for math theorems](https://www.nature.com/articles/s41586-021-04086-x.pdf). The goal is to predict a knot's signature based on its properties. We sampled 10K training and 5K test examples from the [original data](https://github.com/deepmind/mathematics_conjectures/blob/main/knot_theory.ipynb). The sampled dataset make this tutorial run quickly, but AutoGluon can handle the full dataset if desired.

We load this dataset directly from a URL. AutoGluon's `TabularDataset` is a subclass of pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), so any `DataFrame` methods can be used on `TabularDataset` as well.

In [8]:
train_data = TabularDataset(f'/content/drive/MyDrive/AutoGluon_dataset/knot_theory/train.csv')
train_data.head()

,Unnamed: 0,chern_simons,cusp_volume,hyperbolic_adjoint_torsion_degree,hyperbolic_torsion_degree,injectivity_radius,longitudinal_translation,meridinal_translation_imag,meridinal_translation_real,short_geodesic_imag_part,short_geodesic_real_part,Symmetry_0,Symmetry_D3,Symmetry_D4,Symmetry_D6,Symmetry_D8,Symmetry_Z/2 + Z/2,volume,signature
0,70746,0.090530,12.226322,0,10,0.507756,10.685555,1.144192,-0.519157,-2.760601,1.015512,0.0,0.0,0.0,0.0,0.0,1.0,11.393225,-2
1,240827,0.232453,13.800773,0,14,0.413645,10.453156,1.320249,-0.158522,-3.013258,0.827289,0.0,0.0,0.0,0.0,0.0,1.0,12.742782,0
2,155659,-0.144099,14.761030,0,14,0.436928,13.405199,1.101142,0.768894,2.233106,0.873856,0.0,0.0,0.0,0.0,0.0,0.0,15.236505,2
3,239963,-0.171668,13.738019,0,22,0.249481,27.819496,0.493827,-1.188718,-2.042771,0.498961,0.0,0.0,0.0,0.0,0.0,0.0,17.279890,-8
4,90504,0.235188,15.896359,0,10,0.389329,15.330971,1.036879,0.722828,-3.056138,0.778658,0.0,0.0,0.0,0.0,0.0,0.0,16.749298,4


Our targets are stored in the "signature" column, which has 18 unique integers. Even though pandas didn't correctly recognize this data type as categorical, AutoGluon will fix this issue.


In [9]:
train_data.rename(columns={'Unnamed: 0': 'id'}, inplace=True)

In [10]:
train_data.set_index('id')

,chern_simons,cusp_volume,hyperbolic_adjoint_torsion_degree,hyperbolic_torsion_degree,injectivity_radius,longitudinal_translation,meridinal_translation_imag,meridinal_translation_real,short_geodesic_imag_part,short_geodesic_real_part,Symmetry_0,Symmetry_D3,Symmetry_D4,Symmetry_D6,Symmetry_D8,Symmetry_Z/2 + Z/2,volume,signature
id,,,,,,,,,,,,,,,,,,
70746,0.090530,12.226322,0,10,0.507756,10.685555,1.144192,-0.519157,-2.760601,1.015512,0.0,0.0,0.0,0.0,0.0,1.0,11.393225,-2
240827,0.232453,13.800773,0,14,0.413645,10.453156,1.320249,-0.158522,-3.013258,0.827289,0.0,0.0,0.0,0.0,0.0,1.0,12.742782,0
155659,-0.144099,14.761030,0,14,0.436928,13.405199,1.101142,0.768894,2.233106,0.873856,0.0,0.0,0.0,0.0,0.0,0.0,15.236505,2
239963,-0.171668,13.738019,0,22,0.249481,27.819496,0.493827,-1.188718,-2.042771,0.498961,0.0,0.0,0.0,0.0,0.0,0.0,17.279890,-8
90504,0.235188,15.896359,0,10,0.389329,15.330971,1.036879,0.722828,-3.056138,0.778658,0.0,0.0,0.0,0.0,0.0,0.0,16.749298,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197087,-0.146644,15.338084,0,10,0.433621,11.226065,1.366292,-0.363464,-2.974752,0.867243,1.0,0.0,0.0,0.0,0.0,0.0,13.866150,0
192862,-0.228054,14.972895,0,10,0.215976,13.794569,1.085420,0.828377,-2.296227,0.431951,0.0,0.0,0.0,0.0,0.0,0.0,16.346428,4
121923,0.242641,17.499198,0,14,0.481624,16.032106,1.091510,0.642958,-2.986708,0.963248,0.0,0.0,0.0,0.0,0.0,0.0,17.005386,2


In [11]:
label = 'signature'
train_data[label].describe()

,signature
count,10000.000000
mean,-0.022000
std,3.025166
min,-12.000000
25%,-2.000000
50%,0.000000
75%,2.000000
max,12.000000


## Training

We now construct a `TabularPredictor` by specifying the label column name and then train on the dataset with `TabularPredictor.fit()`. We don't need to specify any other parameters. AutoGluon will recognize this is a multi-class classification task, perform automatic feature engineering, train multiple models, and then ensemble the models to create the final predictor.

In [12]:
predictor = TabularPredictor(label=label).fit(train_data,time_limit=600)

No path specified. Models will be saved in: "AutogluonModels/ag-20251015_084945"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          8
Memory Avail:       48.89 GB / 50.99 GB (95.9%)
Disk Space Avail:   190.06 GB / 235.68 GB (80.6%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU memory.
	presets=

Model fitting should take a few minutes or less depending on your CPU. You can make training faster by specifying the `time_limit` argument. For example, `fit(..., time_limit=60)` will stop training after 60 seconds. Higher time limits will generally result in better prediction performance, and excessively low time limits will prevent AutoGluon from training and ensembling a reasonable set of models.



## Prediction

Once we have a predictor that is fit on the training dataset, we can load a separate set of data to use for prediction and evaulation.

In [14]:
test_data = TabularDataset(f'/content/drive/MyDrive/AutoGluon_dataset/knot_theory/test.csv')
test_data.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
test_data.set_index('id')

y_pred = predictor.predict(test_data.drop(columns=[label]))
y_pred.head()

Loaded data from: /content/drive/MyDrive/AutoGluon_dataset/knot_theory/test.csv | Columns = 19 / 19 | Rows = 5000 -> 5000


,signature
0,-4
1,-2
2,0
3,4
4,2


## Evaluation

We can evaluate the predictor on the test dataset using the `evaluate()` function, which measures how well our predictor performs on data that was not used for fitting the models.

In [15]:
predictor.evaluate(test_data, silent=True)

{'accuracy': 0.9498,
 'balanced_accuracy': np.float64(0.7637845709496016),
 'mcc': np.float64(0.9384990771653317)}

AutoGluon's `TabularPredictor` also provides the `leaderboard()` function, which allows us to evaluate the performance of each individual trained model on the test data.

In [16]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.9498,0.960961,accuracy,0.510283,0.092151,16.876954,0.005250,0.000881,0.105809,2,True,12
1,LightGBM,0.9456,0.955956,accuracy,0.171204,0.032744,3.593638,0.171204,0.032744,3.593638,1,True,3
2,XGBoost,0.9448,0.956957,accuracy,0.445883,0.078613,5.327122,0.445883,0.078613,5.327122,1,True,9
3,LightGBMLarge,0.9444,0.949950,accuracy,0.461938,0.074096,7.083533,0.461938,0.074096,7.083533,1,True,11
4,CatBoost,0.9432,0.955956,accuracy,0.028430,0.004816,19.797668,0.028430,0.004816,19.797668,1,True,6
5,RandomForestEntr,0.9384,0.949950,accuracy,0.169662,0.086926,1.845910,0.169662,0.086926,1.845910,1,True,5
6,NeuralNetFastAI,0.9366,0.943944,accuracy,0.059150,0.012658,11.444024,0.059150,0.012658,11.444024,1,True,1
7,ExtraTreesGini,0.9360,0.946947,accuracy,0.217227,0.097252,1.172138,0.217227,0.097252,1.172138,1,True,7
8,ExtraTreesEntr,0.9358,0.942943,accuracy,0.237151,0.096192,1.165220,0.237151,0.096192,1.165220,1,True,8
9,RandomForestGini,0.9352,0.944945,accuracy,0.182209,0.096305,1.779804,0.182209,0.096305,1.779804,1,True,4
